In [1]:
from langchain_groq import ChatGroq

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
llm = ChatGroq(model="moonshotai/kimi-k2-instruct")

In [14]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 32, 'total_tokens': 40, 'completion_time': 0.017735165, 'prompt_time': 0.001257719, 'queue_time': 0.286367372, 'total_time': 0.018992884}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_c5bd0a648b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d755d703-ec4c-474e-95bb-aad99a992b82-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [16]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [17]:
result=embeddings.embed_query("What is the capital of France?")

In [18]:
len(result)

768